In [12]:
!pip install ultralytics kagglehub
!pip install -q roboflow
!pip install -U ultralytics wandb


In [2]:
from ultralytics import YOLO
import os
import cv2 as cv
import matplotlib.pyplot as plt
import kagglehub
import shutil
import wandb
from google.colab import userdata
from huggingface_hub import login, Repository
from roboflow import Roboflow
import gradio as gr

In [10]:
#setting up HF tokens
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

#setting up Weights and biases
wandb_api_key = userdata.get("WANDB_API_KEY")
wandb.login(key=wandb_api_key)

LOG_TO_WANDB=True
PROJECT_NAME = "PCB-Defect-Detection"
RUN_NAME = "YOLOv12-Training"

fine_tuning.model.push_to_hub(PROJECT_RUN_NAME, private=True)
print(f"Saved to the hub: {PROJECT_RUN_NAME}")

# Configure Weights & Biases to record against our project
os.environ["WANDB_PROJECT"] = "PCB-Defect-Detection"
os.environ["WANDB_LOG_MODEL"] = "checkpoint" if LOG_TO_WANDB else "end"
os.environ["WANDB_WATCH"] = "gradients"

if LOG_TO_WANDB:
  wandb.init(project=PROJECT_NAME, name=RUN_NAME)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [19]:
HF_USER = "Varnyaa"
PROJECT_RUN_NAME = "PCB-Defect-Detection"
hf_repo_name = f"{HF_USER}/{PROJECT_RUN_NAME}"


In [5]:
# Load a model
model = YOLO("yolo12n.yaml")  # build a new model from YAML
model = YOLO("yolo12n.pt")  # load a pretrained model (recommended for training)
model = YOLO("yolo12n.yaml").load("yolo12n.pt")  # build from YAML and transfer weights

Transferred 691/691 items from pretrained weights


In [6]:
rf = Roboflow(api_key="an3ukjoeftChxZjp8au3")
project = rf.workspace("sssssc").project("pcb-ryqxj")
version = project.version(3)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...


In [13]:
#wandb.init(project="PCB-Defect-Detection", name="YOLOv12-Training")
!yolo settings wandb=True

JSONDict("/root/.config/Ultralytics/settings.json"):
{
  "settings_version": "0.0.6",
  "datasets_dir": "/content/datasets",
  "weights_dir": "weights",
  "runs_dir": "runs",
  "uuid": "569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb",
  "sync": true,
  "api_key": "",
  "openai_api_key": "",
  "clearml": true,
  "comet": true,
  "dvc": true,
  "hub": true,
  "mlflow": true,
  "neptune": true,
  "raytune": true,
  "tensorboard": true,
  "wandb": true,
  "vscode_msg": true
}
💡 Learn more about Ultralytics Settings at https://docs.ultralytics.com/quickstart/#ultralytics-settings


In [14]:
!yolo task=detect mode=train epochs=25 batch=32 plots=True \
model="/content/yolo11n.pt" \
data='/content/pcb-3/data.yaml' \
project="PCB-Defect-Detection" \
name="YOLOv12"

Ultralytics 8.3.99 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/content/yolo12n.pt, data=/content/pcb-3/data.yaml, epochs=25, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=PCB-Defect-Detection, name=YOLOv123, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, 

In [18]:
!yolo task=detect mode=val conf=0.5 save=True model="/content/PCB-Defect-Detection/YOLOv123/weights/best.pt" \
data='/content/pcb-3/data.yaml' \
split=test \
project="PCB-Defect-Detection" \
name="Test_YOLOv12"


Ultralytics 8.3.99 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv12n summary (fused): 159 layers, 2,558,093 parameters, 0 gradients, 6.3 GFLOPs
val: Scanning /content/pcb-3/test/labels... 79 images, 0 backgrounds, 0 corrupt: 100% 79/79 [00:00<00:00, 2009.84it/s]
val: New cache created: /content/pcb-3/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 5/5 [00:02<00:00,  2.31it/s]
                   all         79        116      0.922      0.972      0.957      0.811
        broken_circuit         12         15      0.929      0.867      0.909      0.713
             burn_down          3          8        0.8          1      0.995      0.964
                damage          5          5          1          1      0.995      0.927
              less_tin          7          7      0.778          1      0.844      0.592
         missing_parts         28         33          1          1      0.995        0.9


In [31]:
from huggingface_hub import HfApi

hf_repo_name = "Varnyaa/PCB-Defect-Detection"  # Change this to your HF username/repo name

api = HfApi()
api.create_repo(repo_id=hf_repo_name, repo_type="model", exist_ok=True)

print(f"✅ Repository created: https://huggingface.co/{hf_repo_name}")


✅ Repository created: https://huggingface.co/Varnyaa/PCB-Defect-Detection


In [48]:
hf_repo_name = "Varnyaa/PCB-Defect-Detection"
local_folder_path = "/content/PCB-Defect-Detection/YOLOv123/weights"

api = HfApi()

# Upload all files in the folder
for root, _, files in os.walk(local_folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        repo_file_path = os.path.relpath(file_path, local_folder_path)  # Keep folder structure
        api.upload_file(
            path_or_fileobj=file_path,
            path_in_repo=repo_file_path,
            repo_id=hf_repo_name,
        )

print(f"✅ Entire folder uploaded to: https://huggingface.co/{hf_repo_name}")

✅ Entire folder uploaded to: https://huggingface.co/Varnyaa/PCB-Defect-Detection


In [54]:
from ultralytics import YOLO

model_path = "/content/PCB-Defect-Detection/YOLOv123/weights/best.pt"
model = YOLO(model_path)

In [56]:
def predict(image):
  result = model.predict(source = image, imgsz = 640, conf = 0.25)
  annotated_img = result[0].plot()
  annotated_img[:,:, ::-1]
  return annotated_img

demo = gr.Interface(
    fn = predict,
    inputs = gr.Image(type="numpy", label="Upload an image"),
    outputs = gr.Image(type="numpy", label="Prediction"),
    title = " PCB Defect Dection using Yolov12",
    description = "Upload an image to detect broken circuit, burn down, damage, less tin, missing parts, more tin and shifting",
)
demo.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://7d2032a998f92bdf0a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://7d2032a998f92bdf0a.gradio.live
